# GPT1 Pretrain model 



## Transformer model , GPT model ( Pretrain ) 비교

- GPT pretrain 의 경우 인코더가 없다
- 디코더에서도 마스크드 멀티 셀프어텐션 부분만을 사용하고 인코더 디코더 멀티헤드 어텐션층은 없다
- 인코더와 인코더-디코더 멀티헤드 어텐션이 없으므로 모델 인풋이 디코더 인풋이 되며 이로 인해 인풋이 달라진다.
- 인풋에서 포지셔널 인코딩 대신 포지셔널 임베딩을 사용하였다.
- GPT pretrain의 경우 비지도 학습이므로 데이터셋의 구성이 달라진다 target을 따로 준비하지 않고 문장의 다음 토큰이 타겟이 됨



## 기존 transformer 코드에서 변경사항
- 인코더, 인코더 레이어 제거
- PositionalEncoding 클래스 -> PositionalEmbedding 클래스
- 디코더 레이어 변경
    - 인코더의 출력을 디코더 레이어에 입력으로 사용하는 부분 제거 ( enc_output 과 padding_mask 제거 )
    - 인코더-디코더 Multihead Attention 제거( 레이어 제거 -> 전차 연결 수정 )
- 디코더 변경
    - 인코더의 출력을 디코더 레이어에 입력으로 사용하는 부분 제거 (enc_outputs, padding_mask 제거)
    - (토큰 임베딩 -> 포지셔널 인코딩) => (토큰 임베딩 + 포지셔널 임베딩)
- transformer 모델 구현을 GPT pretrain 모델로 변경
    - 인코더 디코더 패딩 마스크 제거 ( enc_padding_mask, dec_padding_mask)
    - dec_inputs 제거하고 inputs 로 입력 단일화
    - 인코더 실행 부분 제거
- 예측 문장 생성 함수 변경
    - pretrain 모델에 맞게 현재 받은 입력을 바탕으로 다음 토큰을 예측해 문장을 완성하도록 변경
    - END 토큰을 예측 못할때 반복되는 경우가 많아 반복 제거 코드 추가
- 훈련용 데이터셋 구성 변경 
    - 기존 input, dec_input, output에서 input output으로 변경
    - input 과 output 모두 입력 시퀀스로 input은 처음부터 마지막 바로전까지, output은 두번째부터 마지막까지로 변경
- 모델 layer 수 12개로 변경하려하였다가 데이터가 적을때는 오히려 안좋다고하여 다시 2로 변경
    
 *** 변경 코드에 GPT_QUEST prefix로 주석을 모두 달았으므로 참고 ***

## Import & Define

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pandas as pd
import random

chatbot_data_file = "~/aiffel/transformer_chatbot/data/ChatbotData.csv"

## 클래스 및 함수 정의

### 변경하지 않은 함수 & 제거 함수
- 제거 함수
    - encoder_layer 함수
    - encoder 함수
- 변경하지 않은 함수 ( 생략 아래 참조 )

In [2]:


# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 MAX_LENGTH 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 MAX_LENGTH로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

# 패딩 마스크
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

# 룩어헤드 마스크
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)


# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)    # 매우 작은 값

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs


#모델 훈련용
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

# 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
  sentence = sentence.lower().strip() # 영문이 포함된 문장이 있긴해서 포함

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # 한글 영문 그리고 일부 문자를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z.?!,]+", " ",sentence) 
  sentence = sentence.strip()
  return sentence



### 변경된 코드
 *  변경 부분 GPT_QUEST preifx 로 주석 처리

In [3]:

#GPT_QUEST PositionalEncoding 클래스 -> PositionalEmbedding 클래스
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_position, d_model):
        super(PositionalEmbedding, self).__init__()
        # 포지셔널 임베딩 레이어 생성
        self.position_embedding = tf.keras.layers.Embedding(input_dim=max_position, output_dim=d_model)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        seq_length = tf.shape(inputs)[1]

        # (batch_size, seq_length) 위치 인덱스 생성
        position_indices = tf.tile(
            tf.expand_dims(tf.range(seq_length), 0), [batch_size, 1]
        )

        # 위치 임베딩 조회: (batch_size, seq_length, d_model)
        position_embeddings = self.position_embedding(position_indices)

        return inputs + position_embeddings
    
    
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    
  # GPT_QUEST - enc_outputs, padding_mask 초기화 제거 

  # 첫 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  #  마스크드 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)
    
  # GPT_QUEST 인코더-디코더 어텐션 제거

  # 두 번째 서브 레이어 : 2개의 완전연결층 (피드포워드)
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention1)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention1)       # GPT_QUEST 전차 연결 1번째 레이어 출력으로 변경
                                                
  return tf.keras.Model(
      inputs=[inputs, look_ahead_mask],         # GPT_QUEST 불필요한 enc_outputs, padding_mask 제거
      outputs=outputs,
      name=name)


# 디코더 함수로 구현
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
  # GPT_QUEST 불필요한 enc_outputs, padding_mask 제거

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  #GPT_QUEST 포지셔널 인코딩 => 포지셔널 임베딩
  embeddings = PositionalEmbedding(MAX_LENGTH, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, look_ahead_mask])      # GPT_QUEST 불필요한 enc_output, padding_mask 제거

  return tf.keras.Model(
      inputs=[inputs, look_ahead_mask],       # GPT_QUEST 불필요한 enc_output, padding_mask 제거
      outputs=outputs,
      name=name)

# GPT_QUEST 트랜스포머 모델 -> GPT pretrain 모델로 변경
def GPT_pretrain(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="GPT_pretrain"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  # GPT_QUEST dec_inputs 제거
  #   dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")


  # GPT_QUEST 인코더 디코더 패딩 마스크 제거 ( enc_padding_mask, dec_padding_mask)

  # 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(inputs)

  # GPT_QUEST -  인코더 실행 제거

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, look_ahead_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=inputs, outputs=outputs, name=name)        # GPT_QUEST dec_input 제거


# 문장 예측 출력용 
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # GPT_QUEST - pretrain 모델은 QnA가 아닌 다음 토큰을 예측하는 것이므로 문장 예측 수정
  # 입력된 문장을 토크나이즈하고 START_TOKEN 만을 붙여 이 후 예측 토큰을 확인하도록 변경 
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence), axis=0)

  output_sequence = sentence
  
  past_tokens = set()  # GPT_QUEST - 반복 제거용

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH - tf.shape(sentence)[1]):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=output_sequence, training=False)      # GPT_QUEST output sequece 를 입력으로 변경
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break
    
    # GPT_QUEST 반복 방지: 동일 토큰이 여러 번 나오면 중단 (옵션)
    if int(predicted_id[0][0].numpy()) in past_tokens:
        break
    past_tokens.add(int(predicted_id[0][0].numpy()))
    
    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)


## 데이터 준비
 songys/Chatbot_data
 https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv


In [4]:
data_ori = pd.read_csv(chatbot_data_file, encoding='utf-8')

data_ori.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## 데이터 전처리

- QnA를 합쳐 처리해야하지만 이전 처리 과정들을 스킵하고자 데이터셋 준비 부분에서 Q 와 A를 합쳐서 데이터 구성하도록 변경하였습니다.
- 따라서 이전 코드들은 그대로 사용합니다.


In [5]:
#전처리 

# 결측 데이터 체크
print(data_ori.isnull().sum())

# 데이터를 로드하고 전처리하여 질문을 questions, 답변을 answers에 저장합니다.
questions = [preprocess_sentence(sentence) for sentence in data_ori['Q']]
answers = [preprocess_sentence(sentence) for sentence in data_ori['A']]

print('questions 수:', len(questions))
print('answers 수:', len(answers))

word_cnt = len({word for s in questions + answers for word in s.split()})
print("총 단어수:", word_cnt)

indices = random.sample(range(len(questions)), 3)

print('\n랜덤 Q&A 샘플 3개:\n')
for i in indices:
    print(f"Q({i}): {questions[i]}")
    print(f"A({i}): {answers[i]}\n")


Q        0
A        0
label    0
dtype: int64
questions 수: 11823
answers 수: 11823
총 단어수: 20528

랜덤 Q&A 샘플 3개:

Q(551): 나 좀 안 건들였으면 좋겠어
A(551): 많이 지쳤나봐요 .

Q(3727): 이상한 소문이 돌아
A(3727): 소문은 소문일 뿐이에요 .

Q(10231): 썸 타는데 사랑하다고 말할 수 있음 ?
A(10231): 좋아함을 점프했네요 .



In [6]:
# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

# target_vocab_size 값을 조정하며 테스트 
# target_vocab_size를 작게하는 경우 단어를 여러개의 서브워드로 분리 답변이 제대로 출력되지 못하는 경우 발생
# taget_vovab_size를 크게 하는 경우에는 답변이 더욱 엉뚱해졌다.

In [7]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

START_TOKEN의 번호 : [8141]
END_TOKEN의 번호 : [8142]
8143


In [8]:
# 각 토큰을 고유한 정수로 변환
for i in indices:
    print(f'정수 인코딩 후의 {i}번째 질문 샘플: {tokenizer.encode(questions[i])}')
    print(f'정수 인코딩 후의 {i}번째 답변 샘플: {tokenizer.encode(answers[i])}\n')

정수 인코딩 후의 551번째 질문 샘플: [27, 24, 20, 622, 266, 5030, 299]
정수 인코딩 후의 551번째 답변 샘플: [19, 1491, 1]

정수 인코딩 후의 3727번째 질문 샘플: [1395, 5178, 1040]
정수 인코딩 후의 3727번째 답변 샘플: [2585, 33, 2585, 85, 543, 1]

정수 인코딩 후의 10231번째 질문 샘플: [61, 2185, 1100, 313, 1709, 4, 2494, 2]
정수 인코딩 후의 10231번째 답변 샘플: [140, 2417, 1624, 2177, 3197, 1]



In [9]:
# 각 질문과 답변의 단어 수 계산
questions_lengths = [len(sentence.split()) for sentence in questions]
answers_lengths = [len(sentence.split()) for sentence in answers]

# 최대 길이 찾기
max_question_length = max(questions_lengths)
max_answer_length = max(answers_lengths)

print(f"최대 질문 길이: {max_question_length}")
print(f"최대 답변 길이: {max_answer_length}")

# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 32
print(MAX_LENGTH)

최대 질문 길이: 16
최대 답변 길이: 24
32


In [10]:
# 토크나이징 후 MAX_LENGTH 초과 샘플 제거
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8143
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


### GPT pretrain 관련 훈련 데이터셋 변경
- GPT_QUEST pretrain 모델에 사용하기 위해 Q A 데이터를 하나의 문장 데이터로 합침
    - inputs 는 문장의 처음부터 마지막 이전 토큰까지
    - outputs 는 문장의 두번째부터 마지막 토큰까지로 변경

In [11]:
# GPT_QUEST  Q와 A를 순차적으로 하나씩 sentences에 담기
sentences = np.array([item for pair in zip(questions, answers) for item in pair])


# 데이터를 섞어서 batch size로 분리
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': sentences[:, :-1]
        # GPT_QUEST dec_inputs 제거하고 inputs로 대체
    },
    {
        'outputs': sentences[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## 모델 생성

In [12]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 #  디코더층의 개수
D_MODEL = 256 # 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = GPT_pretrain(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "GPT_pretrain"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 256)    3147008     inputs[0][0]                     
                                                                 look_ahead_mask[0][0]            
__________________________________________________________________________________________________
outputs (Dense)                 (None, None, 8143)   2092751     decoder[0][0]         

In [13]:
#모델 컴파일 

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

## 모델 훈련

In [14]:
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/10
370/370 [==============================] - 14s 30ms/step - loss: 1.7022 - accuracy: 0.0288
Epoch 2/10
370/370 [==============================] - 11s 30ms/step - loss: 1.3526 - accuracy: 0.0477
Epoch 3/10
370/370 [==============================] - 12s 31ms/step - loss: 1.2225 - accuracy: 0.0548
Epoch 4/10
370/370 [==============================] - 12s 32ms/step - loss: 1.1122 - accuracy: 0.0612
Epoch 5/10
370/370 [==============================] - 12s 32ms/step - loss: 1.0040 - accuracy: 0.0695
Epoch 6/10
370/370 [==============================] - 12s 31ms/step - loss: 0.8966 - accuracy: 0.0779
Epoch 7/10
370/370 [==============================] - 11s 31ms/step - loss: 0.7986 - accuracy: 0.0863
Epoch 8/10
370/370 [==============================] - 11s 31ms/step - loss: 0.7118 - accuracy: 0.0956
Epoch 9/10
370/370 [==============================] - 11s 31ms/step - loss: 0.6417 - accuracy: 0.1043
Epoch 10/10
370/370 [==============================] - 11s 31ms/step - loss: 0.589

## 모델 평가

- 완성된 문장을 쓰게되면 다음 토큰을 END로 예측해 문장이 끝나고 해당 문장을 그대로 반환한다.
- 의미상 맞지 않는 문장을 구사하는 경우가 있지만 어느정도 문장을 완성해주긴 한다.

In [15]:
sentence_generation('ㅠㅠ')

입력 : ㅠㅠ
출력 : ㅠㅠ


'ㅠㅠ'

In [16]:
sentence_generation('오늘 뭐했어?')

입력 : 오늘 뭐했어?
출력 : 오늘 뭐했어 ?


'오늘 뭐했어 ?'

In [17]:
sentence_generation('드라마가')

입력 : 드라마가
출력 : 드라마가의 연애에요 .


'드라마가의 연애에요 .'

In [18]:
sentence_generation('영화를')

입력 : 영화를
출력 : 영화를가 재미 없어


'영화를가 재미 없어'

In [19]:
sentence_generation('뭐')

입력 : 뭐
출력 : 뭐부터 바꿔야 할까


'뭐부터 바꿔야 할까'

In [20]:
sentence_generation('음악 ')

입력 : 음악 
출력 : 음악을 들어보세요 .


'음악을 들어보세요 .'

In [21]:
sentence_generation('점심 먹고')

입력 : 점심 먹고
출력 : 점심 먹고는 살아야죠 .


'점심 먹고는 살아야죠 .'

In [22]:
sentence_generation('너무 ')

입력 : 너무 
출력 : 너무힘들어


'너무힘들어'

In [23]:
sentence_generation('오늘은 그것을')

입력 : 오늘은 그것을
출력 : 오늘은 그것을때는 솔직하게 이야기하고 풀어보세요 .


'오늘은 그것을때는 솔직하게 이야기하고 풀어보세요 .'

## 회고

- 현재 사용되는 GPT의 초기 모델에 대해 알수 있어 좋았습니다.
- 출력이 제대로 되지 않아 한참 고생했는데 완벽하게 해결은 못한거 같지만 그래도 이것저것 알아보면서 공부가 되었습니다.
 아직도 파이썬 데이터 구조가 익숙해지지 않은듯하여 더 파이썬을 다뤄봐야겠다고 생각이 들었습니다.
- 특정 토큰이 반복되는 현상이나 깨진 문자 안맞는 단어등이 생성되는데 어떻게 하면 더 잘 처리할 수 있을까 코드상의 문제는 아닐까 생각이 많지만 더 해볼 시간이 없는게 아쉽습니다.
- 실제 GPT pretrain의 경우는 START END 토큰 없이 text 덩어리를 학습한다고 하는데 일단 샘플 데이터가 이어지는 문장들도 아니고해서 START와 END로 문장을 분리해서 학습하고 문장 예측을 하였습니다. START END 토큰 없이 하면 어떤 결과가 나올까 궁금한데 다음에 시간이 나면 해봐야겠습니다.
